<a href="https://www.kaggle.com/code/majessica/eda-sql-animal-shelter-analytics?scriptVersionId=136209757" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## **WIP**: Images will be updated later

# **Exploratory Data Analysis (EDA) with Microsoft SQL Server**
​
After completing *Course 5* of *Google Data Analytics Certificate Program*, I've always wanted to do an *Exploratory Data Analysis* (EDA) to apply what I have learned. This time, I used the [Animal Shelter Analytics](/kaggle/input/animal-shelter-analytics) on Kaggle for my first EDA project.
 
<div id="banner" align="center">
  <img src="https://firehouseaustin.com/wp-content/uploads/2021/09/7.jpg" width="500"/>
</div>
 
As an animal lover, I have a huge respect for people who run and work at the animal rescue centers. After looking through this wholesome dataset, a few questions popped up in my head, and I hope an EDA will help my answer them.
​
***
# About the dataset
​
It contains data from October 1st, 2013 to March 3rd, 2021.
​
​
There are 3 CSV files in the dataset:
- *Austin_Animal_Center_Intake*: Intakes represent the status of animals as they arrive at the Animal Center. All animals receive a unique Animal ID during intake.
​
- *Austin_Animal_Center_Outcome*: Outcomes represent the status of animals as they leave the Animal Center.
​
- *Austin_Animal_Center_Stray_Map*: Map shows all stray cats and dogs that are currently listed in AAC's database for no longer than a week.
​
However, this EDA only concerns the first 2 files. Below are their previews:

In [1]:
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns

intake=pd.read_csv('../input/animal-shelter-analytics/Austin_Animal_Center_Intakes.csv')
outcome=pd.read_csv('../input/animal-shelter-analytics/Austin_Animal_Center_Outcomes.csv')

intake.head()

,Animal ID,Name,DateTime,MonthYear,Found Location,Intake Type,Intake Condition,Animal Type,Sex upon Intake,Age upon Intake,Breed,Color
0,A786884,*Brock,01/03/2019 04:19:00 PM,01/03/2019 04:19:00 PM,2501 Magin Meadow Dr in Austin (TX),Stray,Normal,Dog,Neutered Male,2 years,Beagle Mix,Tricolor
1,A706918,Belle,07/05/2015 12:59:00 PM,07/05/2015 12:59:00 PM,9409 Bluegrass Dr in Austin (TX),Stray,Normal,Dog,Spayed Female,8 years,English Springer Spaniel,White/Liver
2,A724273,Runster,04/14/2016 06:43:00 PM,04/14/2016 06:43:00 PM,2818 Palomino Trail in Austin (TX),Stray,Normal,Dog,Intact Male,11 months,Basenji Mix,Sable/White
3,A665644,NaN,10/21/2013 07:59:00 AM,10/21/2013 07:59:00 AM,Austin (TX),Stray,Sick,Cat,Intact Female,4 weeks,Domestic Shorthair Mix,Calico
4,A682524,Rio,06/29/2014 10:38:00 AM,06/29/2014 10:38:00 AM,800 Grove Blvd in Austin (TX),Stray,Normal,Dog,Neutered Male,4 years,Doberman Pinsch/Australian Cattle Dog,Tan/Gray


In [2]:
outcome.head()

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color
0,A794011,Chunk,05/08/2019 06:20:00 PM,05/08/2019 06:20:00 PM,05/02/2017,Rto-Adopt,NaN,Cat,Neutered Male,2 years,Domestic Shorthair Mix,Brown Tabby/White
1,A776359,Gizmo,07/18/2018 04:02:00 PM,07/18/2018 04:02:00 PM,07/12/2017,Adoption,NaN,Dog,Neutered Male,1 year,Chihuahua Shorthair Mix,White/Brown
2,A821648,NaN,08/16/2020 11:38:00 AM,08/16/2020 11:38:00 AM,08/16/2019,Euthanasia,NaN,Other,Unknown,1 year,Raccoon,Gray
3,A720371,Moose,02/13/2016 05:59:00 PM,02/13/2016 05:59:00 PM,10/08/2015,Adoption,NaN,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff
4,A674754,NaN,03/18/2014 11:47:00 AM,03/18/2014 11:47:00 AM,03/12/2014,Transfer,Partner,Cat,Intact Male,6 days,Domestic Shorthair Mix,Orange Tabby


***
# Table of content

- [Preparation](#task1)
    - [Renaming columns](#task1.1)
    - [Checking for <code>NULL</code> values](#task1.2)
    - [Replacing <code>NULL</code> in <code>outcome_type</code> column to <code>Unknown</code>](#task1.3)
    
    
- [When did the animals normally arrive?](#task2)
    - [By Year](#task2.1)
    - [By Month](#task2.2)
    - [By Month-Year](#task2.3)
    
    
- [Names of the animals arrived](#task3)
    - [Percentage of animals with vs. without a name](#task3.1)
    - [Percentage of animals with vs. without a name by animal type](#task3.2)
    - [Top 3 most common names for each animal type](#task3.3)
    
    
- [Intake types and their frequencies](#task4)

- [Number of cases per intake type per animal type](#task5)

- [Top 5 most common breeds by animal type](#task6)

- [Neuter status upon intake vs outcome](#task7)

- [When did the animals normally leave?](#task8)
    - [By Year](#task8.1)
    - [By Month](#task8.2)
    - [By Month-Year](#task8.3)


- [Outcome types and their frequencies](#task9)

- [Number of cases per outcome type per animal type](#task10)

- [Top 5 most popular breed adopted](#task11)

***
***
# Data analysis process

Firstly, I wrote the query below to have a closer look at the dataset that I would be working with.

```ruby
SELECT
	*
FROM
	portfolio_project..animal_intake

SELECT
	*
FROM
	portfolio_project..animal_outcome;
```

***
## Preperation <a class="anchor"  id="task1"></a>

### Renaming columns as they are inconsistent, but I'm only gonna change columns that I will use for my EDA <a class="anchor"  id="task1.1"></a>

**Intake table**

```ruby
EXEC sp_rename 'portfolio_project..animal_intake.Animal_ID', 'id', 'COLUMN'
EXEC sp_rename 'portfolio_project..animal_intake.Name', 'name', 'COLUMN'
EXEC sp_rename 'portfolio_project..animal_intake.DateTime', 'date_time', 'COLUMN'
EXEC sp_rename 'portfolio_project..animal_intake.Intake_Type', 'intake_type', 'COLUMN'
EXEC sp_rename 'portfolio_project..animal_intake.Intake_Condition', 'intake_condition', 'COLUMN'
EXEC sp_rename 'portfolio_project..animal_intake.Animal_Type', 'animal_type', 'COLUMN'
EXEC sp_rename 'portfolio_project..animal_intake.Sex_upon_Intake', 'sex_upon_intake', 'COLUMN'
EXEC sp_rename 'portfolio_project..animal_intake.Breed', 'breed', 'COLUMN';
```

**Outcome table**

```ruby
EXEC sp_rename 'portfolio_project..animal_outcome.Animal_ID', 'id', 'COLUMN'
EXEC sp_rename 'portfolio_project..animal_outcome.Name', 'name', 'COLUMN'
EXEC sp_rename 'portfolio_project..animal_outcome.DateTime', 'date_time', 'COLUMN'
EXEC sp_rename 'portfolio_project..animal_outcome.Outcome_Type', 'outcome_type', 'COLUMN'
EXEC sp_rename 'portfolio_project..animal_outcome.Animal_Type', 'animal_type', 'COLUMN'
EXEC sp_rename 'portfolio_project..animal_outcome.Sex_upon_Outcome', 'sex_upon_outcome', 'COLUMN'
EXEC sp_rename 'portfolio_project..animal_outcome.Breed', 'breed', 'COLUMN';
```

<div align="center">
<a href="https://imgbb.com/"><img src="https://i.ibb.co/qk8V81Z/in-rename.png" alt="intake_renamed" border="0"></a><br>
<a href="https://imgbb.com/"><img src="https://i.ibb.co/VJ0tqKF/out-rename.png" alt="outcome_renamed" border="0"></a><br>
    <i>After modification</i>
</div>

### Checking for <code>NULL</code> values <a class="anchor"  id="task1.2"></a>

I saw some <code>NULL</code>s as I was scrolling through the dataset, so among the renamed columns, I checked where they were and if I should remove or replace them.

**Intake table**

Only the <code>name</code> column has <code>NULL</code> values, which I believe to not affect the overall analysis so I kept them as they are.

```ruby
SELECT 
  SUM(CASE WHEN id IS NULL THEN 1 ELSE 0 END) AS id_null,
  SUM(CASE WHEN name IS NULL THEN 1 ELSE 0 END) AS name_null,
  SUM(CASE WHEN date_time IS NULL THEN 1 ELSE 0 END) AS date_null,
  SUM(CASE WHEN intake_type IS NULL THEN 1 ELSE 0 END) AS in_type_null,
  SUM(CASE WHEN intake_condition IS NULL THEN 1 ELSE 0 END) AS in_condition_null,
  SUM(CASE WHEN animal_type IS NULL THEN 1 ELSE 0 END) AS ani_type_null,
  SUM(CASE WHEN sex_upon_intake IS NULL THEN 1 ELSE 0 END) AS sex_null,
  SUM(CASE WHEN breed IS NULL THEN 1 ELSE 0 END) AS breed_null
FROM
	portfolio_project..animal_intake;
```

<div align="center">
<a href="https://imgbb.com/"><img src="https://i.ibb.co/3T6BNXs/null-intake.png" alt="null-intake" border="0"></a><br>
    <i>Number of <code>NULL</code>s in each column</i>
</div>

**Outcome table**

After checking, <code>name</code> column have <code>NULL</code> values, which are not important in my opinion so I decided to keep them untouched as well.

However, I planned to do some analysis with <code>outcome_type</code> so I wanted to change <code>NULL</code> values into <code>Unknown</code>.

```ruby
SELECT 
  SUM(CASE WHEN id IS NULL THEN 1 ELSE 0 END) AS id_null,
  SUM(CASE WHEN name IS NULL THEN 1 ELSE 0 END) AS name_null,
  SUM(CASE WHEN date_time IS NULL THEN 1 ELSE 0 END) AS date_null,
  SUM(CASE WHEN outcome_type IS NULL THEN 1 ELSE 0 END) AS out_type_null,
  SUM(CASE WHEN animal_type IS NULL THEN 1 ELSE 0 END) AS ani_type_null,
  SUM(CASE WHEN sex_upon_outcome IS NULL THEN 1 ELSE 0 END) AS sex_null,
  SUM(CASE WHEN age_upon_outcome IS NULL THEN 1 ELSE 0 END) AS age_null,
  SUM(CASE WHEN breed IS NULL THEN 1 ELSE 0 END) AS breed_null
FROM
	portfolio_project..animal_outcome;
```

<div align="center">
<a href="https://imgbb.com/"><img src="https://i.ibb.co/Lk8pKmF/null-outcome.png" alt="null-outcome" border="0"></a><br>
    <i>Number of <code>NULL</code>s in each column</i>
</div>

### Replacing <code>NULL</code> in <code>outcome_type</code> column with <code>Unknown</code> <a class="anchor"  id="task1.3"></a>

```ruby
## Replacing NULL with 'Unknown'
UPDATE
    portfolio_project..animal_outcome
SET
	outcome_type = 'Unknown'
WHERE
	outcome_type IS NULL;

## Checking if all NULLs are replaced
SELECT
	*
FROM
	portfolio_project..animal_outcome
WHERE
	outcome_type LIKE 'Unknown';
```

<div align="center">
<a href="https://imgbb.com/"><img src="https://i.ibb.co/9cbByh1/null-outcome-mod.png" alt="null-outcome_mod" border="0"></a><br>
    <i>No more <code>NULL</code> in <code>outcome_type</code></i>
</div>

***
## Intakes that Austin Animal Center received <a class="anchor"  id="task2"></a>

### By Year <a class="anchor"  id="task2.1"></a>

AAC had the highest intake count in **2019**, which was *19.728* intakes.

```ruby
SELECT
	YEAR(date_time) AS year_of_intake,
	COUNT(*) AS no_of_intake
FROM
	portfolio_project..animal_intake
GROUP BY
	YEAR(date_time)
ORDER BY
	no_of_intake DESC;
```

<div align="center">
<a href="https://imgbb.com/"><img src="https://i.ibb.co/tXZTKKm/in-year.png" alt="intake_year" border="0"></a><br>
</div>

### By Month <a class="anchor"  id="task2.2"></a>

AAC had the highest intake count in **May**, which was *12.337* intakes. **April** came pretty close at *12.107*.

```ruby
SELECT
	MONTH(date_time) AS month_of_intake,
	COUNT(*) AS no_of_intake
FROM
	portfolio_project..animal_intake
GROUP BY
	MONTH(date_time)
ORDER BY
	no_of_intake DESC;
```

<div align="center">
<a href="https://imgbb.com/"><img src="https://i.ibb.co/7jfXJwW/in-mo.png" alt="intake_mo" border="0"></a><br>
</div>

### By Month-Year <a class="anchor"  id="task2.3"></a>

AAC had the highest intake count in **June 2015** and **May 2019**, which were *2.189* and *2.132* intakes respectively.

```ruby
SELECT
	CASE
		WHEN LEN(MONTH(date_time)) = 1
			THEN CONCAT('0', MONTH(date_time), '-', YEAR(date_time)) 
			ELSE CONCAT(MONTH(date_time), '-', YEAR(date_time))
	END AS time_of_intake,
	COUNT(*) AS no_of_intake
FROM
	portfolio_project..animal_intake
GROUP BY
	MONTH(date_time), YEAR(date_time)
ORDER BY
	no_of_intake DESC;
```

<div align="center">
<a href="https://imgbb.com/"><img src="https://i.ibb.co/D8vFftX/in-mmyy.png" alt="intake_mmyy" border="0"></a><br>
    <i>First 10 results</i>
</div>

***
## Name of the animals arrived <a class="anchor"  id="task3"></a>

### Percentage of animals with vs. without a name <a class="anchor"  id="task3.1"></a>

*85.069* (*68,54%*) animals arrived **with a name**, while the other *39.051* (*31,64%*) came in **without a name**.

I used subquery to make the calculations look cleaner. For some reasons, I had to convert the <code>with_name</code> and <code>without_name</code> columns into <code>FLOAT</code> data type for the percentage calculation to work. If not, it would have just returned <code>0</code> *(I figured this out thanks to Google Bard)*.

```ruby
SELECT
  with_name,
  total_animals,
  CONCAT(ROUND(CAST(with_name AS FLOAT) / total_animals * 100, 2), '%') AS percent_with_name,
  without_name,
  CONCAT(ROUND(CAST(without_name AS FLOAT) / total_animals * 100, 2), '%') AS percent_without_name
FROM
  (
  SELECT
    SUM(CASE WHEN name IS NOT NULL THEN 1 ELSE 0 END) AS with_name,
    SUM(CASE WHEN name IS NULL THEN 1 ELSE 0 END) AS without_name,
    COUNT(*) AS total_animals
  FROM
    portfolio_project..animal_intake
  ) AS data_for_calc;
```

<div align="center">
<a href="https://imgbb.com/"><img src="https://i.ibb.co/gvgM9dW/name-percent.png" alt="name_percent" border="0"></a><br>
</div>

### Percentage of animals with vs. without a name by animal type <a class="anchor"  id="task3.2"></a>

**Dogs** had the most animals came in **with a name** (*82,85%*), they were also the **most common** animal at the Animal Center with *70.447* animals. On the other hand, only *9,09%* of **livestocks had a name** upon arrival, they were the **least common** type as well with only *22* entries.

```ruby
SELECT
	animal_type,
	total_animals,
	with_name,
	CONCAT(ROUND(CAST(with_name AS FLOAT) / total_animals * 100, 2), '%') AS percent_with_name,
	without_name,
	CONCAT(ROUND(CAST(without_name AS FLOAT) / total_animals * 100, 2), '%') AS percent_without_name
FROM
	(
	SELECT
		animal_type,
		SUM(CASE WHEN name IS NOT NULL THEN 1 ELSE 0 END) AS with_name,
		SUM(CASE WHEN name IS NULL THEN 1 ELSE 0 END) AS without_name,
		COUNT(*) AS total_animals
	FROM
		portfolio_project..animal_intake
	GROUP BY
		animal_type
	) AS data_for_calc
ORDER BY
	percent_with_name DESC;
```

<div align="center">
<a href="https://imgbb.com/"><img src="https://i.ibb.co/G5XbmfH/name-percent-type.png" alt="name_percent_type" border="0"></a><br>
</div>

### Top 3 most common names for each animal type <a class="anchor"  id="task3.3"></a>

I was curious to know which are the 3 most common names for each animal types so I wrote this query:

```ruby
WITH top3 AS
(
    SELECT
		animal_type,
		name,
		COUNT(*) AS total_count,
		ROW_NUMBER() OVER
			(
			PARTITION BY
				animal_type
			ORDER BY
				COUNT(*) DESC
			)
		AS rank 
    FROM 
        portfolio_project..animal_intake
	WHERE
		name IS NOT NULL
	GROUP BY
		animal_type,
		name
)

SELECT
	animal_type,
	name,
	total_count
FROM
	top3
WHERE
	rank <= 3;
```

<div align="center">
<a href="https://imgbb.com/"><img src="https://i.ibb.co/DbdrC1s/top3-name.png" alt="top3_name" border="0"></a><br>
    <i>Top 3 names per animal type with their respective repeats</i>
</div>

As there are only 2 livestocks that were named so the results only returned *2* names (aka top *2*).

For birds, since the third result has the total count of *1*, so the top 3 result is not quite correct if there were many birds with unique names. I wrote the query below to check for that.

```ruby
SELECT
	COUNT(*) AS bird_unique_name
FROM
	(
	SELECT
		animal_type,
		name,
		COUNT(*) AS total_count
	FROM
		portfolio_project..animal_intake
	WHERE
		animal_type LIKE 'Bird'
		AND name IS NOT NULL
	GROUP BY
		animal_type,
		name
	HAVING
		COUNT(*) = 1
	) AS bird_unique_name;
```

<div align="center">
<a href="https://imgbb.com/"><img src="https://i.ibb.co/48NkWRk/bird-1.png" alt="bird_unique_name" border="0"></a><br>
    <i>There are <b>122</b> birds with unique names</i>
</div>

***
## Intake types and their frequencies <a class="anchor"  id="task4"></a>

The most common method of intake is **Stray** with *86.455* cases.

```ruby
SELECT
	intake_type,
	COUNT(*) AS total_number
FROM
	portfolio_project..animal_intake
GROUP BY
	intake_type
ORDER BY
	total_number DESC;
```

<div align="center">
<a href="https://imgbb.com/"><img src="https://i.ibb.co/5YKqc2Z/intake-type.png" alt="intake_type" border="0"></a><br>
    <i>There are <b>6</b> different intake types</i>
</div>

***
## Number of cases of each intake type per animal type <a class="anchor"  id="task5"></a>

Most animal types have **Stray** as the most frequent intake type, except for <code>Other</code> which has **Wildlife** as the most frequent one. There are also no records of **Wildlife** case for
<code>Cat</code>, <code>Dog</code> and <code>Livestock</code>.

```ruby
SELECT
	animal_type,
	intake_type,
	COUNT(*) AS total_intakes
FROM
	portfolio_project..animal_intake
GROUP BY
	animal_type,
	intake_type
ORDER BY
	animal_type,
	total_intakes DESC;
```

<div align="center">
<a href="https://imgbb.com/"><img src="https://i.ibb.co/D7Xhzwd/intake-type-ani.png" alt="intake_type_ani" border="0"></a><br>
</div>

***
## Top 5 most common breeds by animal type <a class="anchor" id="task6"></a>

The same issue in the top 3 name analysis happens here where the top 5 <code>Livestock</code> result only has 1 record. This indicates that after the first 4 results, the rest livestock breeds are unique.

```ruby
WITH top5 AS
(
    SELECT
        animal_type,
        breed,
        COUNT(*) AS total_count,
        ROW_NUMBER() OVER
            (
            PARTITION BY
                animal_type
            ORDER BY
                COUNT(*) DESC
            )
        AS rank
    FROM
        portfolio_project..animal_intake
    GROUP BY
        animal_type,
        breed
)

SELECT
    animal_type,
    breed,
    total_count
FROM
    top5
WHERE
    rank <= 5;
```

<div align="center">
<a href="https://imgbb.com/"><img src="https://i.ibb.co/HGBVnFB/top5-breed.png" alt="top5_breed" border="0"></a><br>
</div>

***
## Neuter status upon intake vs outcome <a class="anchor"  id="task7"></a>

The number of intact males upon **intake** were more than 2 times higher than that of castrated males, with *40.316* vs. *19.142* animals respectively. The same goes for the female animals with *38.118* intact females compared to *16.310* spayed females.

Regarding **outcome** status, the number between intact and desexed got reversed as there were *15.786* intact vs. *43.794* neutered males, and *15.329* intact vs. *39.359* spayed females.

```ruby
## Intake
SELECT
	sex_upon_outcome,
	COUNT(*) AS total_count
FROM
	portfolio_project..animal_outcome
GROUP BY
	sex_upon_outcome
ORDER BY
	total_count DESC;

## Outcome
SELECT
	sex_upon_intake,
	COUNT(*) AS total_count
FROM
	portfolio_project..animal_intake
GROUP BY
	sex_upon_intake
ORDER BY
	total_count DESC;
```

<div align="center">
<a href="https://imgbb.com/"><img src="https://i.ibb.co/n7T2Yd7/after-sex.png" alt="intake_sex" border="0"></a>&nbsp&nbsp&nbsp&nbsp&nbsp
<a href="https://imgbb.com/"><img src="https://i.ibb.co/K0jHkwy/distinct-sex.png" alt="outcome_sex" border="0"></a><br>
    <i>The total count of the animal's sex upon intake and outcome</i>
</div>

***
## When did the animals normally leave? <a class="anchor"  id="task8"></a>

### By Year <a class="anchor"  id="task8.1"></a>

A
AC had the highest outcome count in **2019**, which reached *19.774*.

```ruby
SELECT
	YEAR(date_time) AS year_of_outcome,
	COUNT(*) AS no_of_outcome
FROM
	portfolio_project..animal_outcome
GROUP BY
	YEAR(date_time)
ORDER BY
	no_of_outcome DESC;
```

<div align="center">
<a href="https://imgbb.com/"><img src="https://i.ibb.co/f9Rq4MM/out-year.png" alt="out_year" border="0"></a><br>
</div>

### By Month <a class="anchor"  id="task8.2"></a>

AAC had the highest outcome count in **October**, which was *11.815* outcomes. **July** came pretty close at *11.802*.

```ruby
SELECT
	MONTH(date_time) AS month_of_outcome,
	COUNT(*) AS no_of_outcome
FROM
	portfolio_project..animal_outcome
GROUP BY
	MONTH(date_time)
ORDER BY
	no_of_outcome DESC;
```

<div align="center">
<a href="https://imgbb.com/"><img src="https://i.ibb.co/p1yWR8V/out-mo.png" alt="out_mm" border="0"></a><br>
</div>

### By Month-Year <a class="anchor"  id="task8.3"></a>

AAC had the highest outcome count in **June 2019**, which were *2.243* outcomes.

```ruby
SELECT
	CASE
		WHEN LEN(MONTH(date_time)) = 1
			THEN CONCAT('0', MONTH(date_time), '-', YEAR(date_time)) 
			ELSE CONCAT(MONTH(date_time), '-', YEAR(date_time))
	END AS time_of_outcome,
	COUNT(*) AS no_of_outcome
FROM
	portfolio_project..animal_outcome
GROUP BY
	MONTH(date_time), YEAR(date_time)
ORDER BY
	no_of_outcome DESC;
```

<div align="center">
<a href="https://imgbb.com/"><img src="https://i.ibb.co/72dCVt6/out-mmyy.png" alt="out_mmyy" border="0"></a><br>
    <i>First 10 results</i>
</div>

***
## Outcome types and their frequencies <a class="anchor"  id="task9"></a>

The most common method of intake is **Adoption** with *55.347* cases.

```ruby
SELECT
	outcome_type,
	COUNT(*) AS total_number
FROM
	portfolio_project..animal_outcome
GROUP BY
	outcome_type
ORDER BY
	total_number DESC;
```

<div align="center">
<a href="https://imgbb.com/"><img src="https://i.ibb.co/Js5cFmN/out-type.png" alt="out_type" border="0"></a><br>
    <i>There are <b>10</b> different outcome types</i>
</div>

***
## Number of cases per outcome type per animal type <a class="anchor"  id="task10"></a>

The most common outcome for most animal types is **Adoption**, where as for <code>Other</code> category, the most common one is **Euthanasia**, sadly. The rate of <code>Cat</code>s being **transfer**red comes pretty close second, *20.273* cases compared to *21.391* adoption cases.


```ruby
SELECT
	animal_type,
	intake_type,
	COUNT(*) AS total_intakes
FROM
	portfolio_project..animal_intake
GROUP BY
	animal_type,
	intake_type
ORDER BY
	animal_type,
	total_intakes DESC;
```

<div align="center">
<a href="https://imgbb.com/"><img src="https://i.ibb.co/sQYcLqz/out-type-ani1.png" alt="out_type_ani1" border="0"></a>&nbsp&nbsp&nbsp&nbsp&nbsp
<a href="https://imgbb.com/"><img src="https://i.ibb.co/mzvh5TJ/out-type-ani2.png" alt="out_type_ani2" border="0"></a><br>
</div>

***
## Top 5 most popular breed adopted <a class="anchor"  id="task11"></a>

There were only 8 adoptions for <code>Livestock</code>, so even when the top 2-5 only have 1 case each, the total count is still 8.

```ruby
WITH top5_adopted AS
(
    SELECT
        animal_type,
        breed,
        COUNT(*) AS total_count,
        ROW_NUMBER() OVER
            (
            PARTITION BY
                animal_type
            ORDER BY
                COUNT(*) DESC
            )
        AS rank
    FROM
        portfolio_project..animal_outcome
	WHERE
		outcome_type LIKE 'Adoption'
    GROUP BY
        animal_type,
        breed
)

SELECT
    animal_type,
    breed,
    total_count
FROM
    top5_adopted
WHERE
    rank <= 5;
```

<div align="center">
<a href="https://imgbb.com/"><img src="https://i.ibb.co/XYqtS8K/top5-breed-adopt.png" alt="top5_breed_adopt" border="0"></a><br>
</div>

***
### That is also the end of this project! Thank you for reading till the end.

<div align="right">
    Hope you have a great day and Happy Coding! :)
</div>